# Chapter 5 - 2

## 검증 세트
- 훈련세트를 나눠서 검증 세트로, 훈련 세트에서 모델을 훈련하고 검증 세트로 모델 평가
- 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다.
- 훈련 세트와 검정 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다
- 마지막에 테스트 세트에서 최종 점수를 평가한다

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# class열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

- train_input과 train_target을 다시 train_test_split() 함수에 넣어서 훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target을 만든다.
- test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input으로 만든다.

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
# 훈련 세트와 검증 세트의 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
# 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 과대적합

## 교차 검증

- 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하고 이 점수를 평균하여 최종 검증 점수를 얻는다.

- 사이킷런에는 cross_validate()라는 교차 검증 함수가 있다.
- 평가할 모델 객체를 첫 번째 매개변수로 전달하고 훈련 세트 전체를 전달
- fit_time, score_time, test_score 키를 가진 딕셔너리 반환
- 처음 2개의 키는 각각 모델을 훈련하는 시간, 검증하는 시간, 키 마다 5개의 숫자가 담겨 있다.
- cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다.
- cv 매개변수에서 폴드 수를 바꿀 수도 있다.

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00861359, 0.0059793 , 0.00697637, 0.0069766 , 0.00598001]), 'score_time': array([0.00098944, 0.00099707, 0.        , 0.        , 0.00099659]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다, 검증 폴드의 점수

In [9]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


- cross_validate()는 훈련 세트를 섞어서 폴드를 나누지 않음, 훈련 세트를 섞으려면 분할기를 지정해야함
- 회귀 모델일 경우 KFold 분할기, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용

In [10]:
# 앞에서 수행한 교차검증과 동일
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행
# n_splits 매개변수는 몇(k) 폴드 교차 검증을 랄지 정함
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

- 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터: 하이퍼파라미터
- 머신러닝 라이브러리를 사용할 떄 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현됨
- 라이브러리가 제공하는 기본값으로 모델 훈련 > 검증 세트의 점수나 교차 검증을 통해 매개변수 조금씩 바꾸기
- 매개변수가 여러 개 있을 때 동시에 바꿔가며 최적의 값을 찾아야함

- 사이킷런에서 제공하는 그리드 서치 사용
- GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
- cross_validate() 함수를 호출할 필요없음
- 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기

In [14]:
# GridSerchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.
# 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리트 서치 객체를 만듦
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

- gs 객체에 fit() 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행
- cv 매개변수 기본값은 5이므로 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다.
- 5 x 5 = 25개의 모델을 훈련함
- n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정한다. 기본값 1. -1로 지정하면 시스템에 있는 모든 코어 사용

In [17]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델을 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
- 이 모델은 gs 객체의 best_estimater_ 속성에 저장

In [18]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


- 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장

In [19]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


- 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장돼있음

In [21]:
# 5번의 교차 검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- numpy의 argmax() 함수: 가장 큰 값의 인덱스를 추출할 수 있다.
- 인덱스를 사용해 params 키에 저장된 매개변수를 출력

In [22]:
# 최상의 검증 점수를 만든 매개변수 조합
# gs.best_params_와 동일한지 확인해보기
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#### 1. 탐색할 매개변수 지정
#### 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾기
#### 3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련

- 결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.
- max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수 고르기

In [23]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          } # 252

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [25]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## 랜덤 서치

In [27]:
from scipy.stats import uniform, randint

In [28]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 7, 1, 2, 5, 8, 8, 0, 0, 7], dtype=int64)

In [29]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([107,  80,  84,  72, 121,  96, 113, 111, 124,  92], dtype=int64))

In [30]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.43313623, 0.58684816, 0.51218803, 0.41135232, 0.26324546,
       0.52134712, 0.66422562, 0.06083255, 0.991619  , 0.41882999])

In [31]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [32]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002913327E550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002913327DDD0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002913327ED10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002913327EA10>},
                   random_state=42)

In [33]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [35]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86
